# Vector Search Configuration Decision Tree

**Purpose**: Analyze your data to make two key decisions for Databricks Vector Search:

1. **Embedding Strategy** (Sections 1-10): Full text vs. chunking
2. **Filter Column Selection** (Sections 11-15): Which columns to index for filtering

---

# Part 1: Embedding Strategy (Chunking Decision)

## Decision Framework

```
                    ┌─────────────────────────────────┐
                    │  Analyze Your Text Data         │
                    └───────────────┬─────────────────┘
                                    │
                    ┌───────────────▼─────────────────┐
                    │  Avg Token Count per Document   │
                    └───────────────┬─────────────────┘
                                    │
              ┌─────────────────────┼─────────────────────┐
              │                     │                     │
      ┌───────▼───────┐     ┌───────▼───────┐     ┌───────▼───────┐
      │   < 256       │     │  256 - 512    │     │    > 512      │
      │   tokens      │     │   tokens      │     │   tokens      │
      └───────┬───────┘     └───────┬───────┘     └───────┬───────┘
              │                     │                     │
              ▼                     ▼                     ▼
      ┌───────────────┐     ┌───────────────┐     ┌───────────────┐
      │ EMBED FULL    │     │ EMBED FULL    │     │ CHUNK TEXT    │
      │ TEXT          │     │ (monitor      │     │ Required      │
      │ ✅ Optimal    │     │ truncation)   │     │               │
      └───────────────┘     └───────────────┘     └───────────────┘
```

## Key Thresholds

| Embedding Model | Max Tokens | Safe Zone | Chunking Recommended |
|-----------------|------------|-----------|----------------------|
| `databricks-bge-large-en` | 512 | < 400 | > 512 |
| `databricks-gte-large-en` | 512 | < 400 | > 512 |
| OpenAI `text-embedding-3-small` | 8191 | < 6000 | > 8000 |
| Cohere `embed-english-v3` | 512 | < 400 | > 512 |


## 1. Configuration


In [ ]:
# Configuration - Update these for your table
CATALOG = "juan_use1_catalog"
SCHEMA = "retail"
TABLE_NAME = "gold_customer_reviews"
TEXT_COLUMN = "review_text"  # Column to be embedded

FULL_TABLE_NAME = f"{CATALOG}.{SCHEMA}.{TABLE_NAME}"

# Embedding model constraints (Databricks BGE-Large-EN)
EMBEDDING_MODEL = "databricks-bge-large-en"
MAX_TOKENS = 512           # Model's maximum token limit
SAFE_TOKEN_THRESHOLD = 400 # Leave buffer for special tokens
CHARS_PER_TOKEN = 4        # Approximate (English text average)

# Chunking parameters (if needed)
CHUNK_SIZE = 400           # Tokens per chunk
CHUNK_OVERLAP = 50         # Token overlap between chunks

print(f"Analyzing: {FULL_TABLE_NAME}")
print(f"Text Column: {TEXT_COLUMN}")
print(f"Embedding Model: {EMBEDDING_MODEL} (max {MAX_TOKENS} tokens)")


## 2. Text Length Analysis (Characters)


In [ ]:
# Analyze character length distribution
char_stats_df = spark.sql(f"""
    SELECT 
        COUNT(*) as total_documents,
        MIN(LENGTH({TEXT_COLUMN})) as min_chars,
        MAX(LENGTH({TEXT_COLUMN})) as max_chars,
        ROUND(AVG(LENGTH({TEXT_COLUMN})), 0) as avg_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.25) as p25_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.50) as median_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.75) as p75_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.90) as p90_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.95) as p95_chars,
        PERCENTILE(LENGTH({TEXT_COLUMN}), 0.99) as p99_chars
    FROM {FULL_TABLE_NAME}
    WHERE {TEXT_COLUMN} IS NOT NULL
""")

stats = char_stats_df.collect()[0]

print("=" * 60)
print("CHARACTER LENGTH DISTRIBUTION")
print("=" * 60)
print(f"\nTotal Documents: {stats['total_documents']:,}")
print(f"\nBasic Statistics:")
print(f"  Min:    {int(stats['min_chars']):,} chars")
print(f"  Max:    {int(stats['max_chars']):,} chars")
print(f"  Avg:    {int(stats['avg_chars']):,} chars")
print(f"  Median: {int(stats['median_chars']):,} chars")
print(f"\nPercentile Distribution:")
print(f"  P25:  {int(stats['p25_chars']):,} chars")
print(f"  P50:  {int(stats['median_chars']):,} chars")
print(f"  P75:  {int(stats['p75_chars']):,} chars")
print(f"  P90:  {int(stats['p90_chars']):,} chars")
print(f"  P95:  {int(stats['p95_chars']):,} chars")
print(f"  P99:  {int(stats['p99_chars']):,} chars")

display(char_stats_df)


## 3. Token Estimation

**Rule of Thumb**: For English text, approximately 1 token ≈ 4 characters (or ~0.75 words)

This is an approximation. For exact counts, use a tokenizer like `tiktoken` or the model's native tokenizer.


In [ ]:
# Estimate token counts from character lengths
token_stats_df = spark.sql(f"""
    SELECT 
        -- Character stats
        LENGTH({TEXT_COLUMN}) as char_count,
        -- Estimated token count (chars / 4)
        ROUND(LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN}, 0) as est_tokens,
        -- Word count for reference
        SIZE(SPLIT({TEXT_COLUMN}, ' ')) as word_count
    FROM {FULL_TABLE_NAME}
    WHERE {TEXT_COLUMN} IS NOT NULL
""")

# Calculate token distribution
token_agg_df = token_stats_df.selectExpr(
    "COUNT(*) as total_docs",
    "MIN(est_tokens) as min_tokens",
    "MAX(est_tokens) as max_tokens",
    "ROUND(AVG(est_tokens), 0) as avg_tokens",
    "PERCENTILE(est_tokens, 0.50) as median_tokens",
    "PERCENTILE(est_tokens, 0.90) as p90_tokens",
    "PERCENTILE(est_tokens, 0.95) as p95_tokens",
    "PERCENTILE(est_tokens, 0.99) as p99_tokens",
    f"SUM(CASE WHEN est_tokens > {MAX_TOKENS} THEN 1 ELSE 0 END) as exceeds_max",
    f"SUM(CASE WHEN est_tokens > {SAFE_TOKEN_THRESHOLD} THEN 1 ELSE 0 END) as exceeds_safe"
)

token_stats = token_agg_df.collect()[0]

print("=" * 60)
print("ESTIMATED TOKEN DISTRIBUTION")
print("=" * 60)
print(f"\nToken Statistics (estimated at {CHARS_PER_TOKEN} chars/token):")
print(f"  Min:    {int(token_stats['min_tokens']):,} tokens")
print(f"  Max:    {int(token_stats['max_tokens']):,} tokens")
print(f"  Avg:    {int(token_stats['avg_tokens']):,} tokens")
print(f"  Median: {int(token_stats['median_tokens']):,} tokens")
print(f"  P90:    {int(token_stats['p90_tokens']):,} tokens")
print(f"  P95:    {int(token_stats['p95_tokens']):,} tokens")
print(f"  P99:    {int(token_stats['p99_tokens']):,} tokens")

print(f"\n⚠️  Model Limit Analysis ({EMBEDDING_MODEL}):")
print(f"  Documents exceeding {MAX_TOKENS} tokens (MAX):  {int(token_stats['exceeds_max']):,}")
print(f"  Documents exceeding {SAFE_TOKEN_THRESHOLD} tokens (SAFE): {int(token_stats['exceeds_safe']):,}")

exceeds_pct = (token_stats['exceeds_max'] / token_stats['total_docs']) * 100
print(f"\n  Truncation Risk: {exceeds_pct:.2f}% of documents")

display(token_agg_df)


## 4. Token Distribution Histogram


In [ ]:
# Create token buckets for visualization
bucket_df = spark.sql(f"""
    SELECT 
        CASE 
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 50 THEN '0-50'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 100 THEN '50-100'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 150 THEN '100-150'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 200 THEN '150-200'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 256 THEN '200-256'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 400 THEN '256-400 (SAFE)'
            WHEN LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN} < 512 THEN '400-512 (CAUTION)'
            ELSE '512+ (TRUNCATED)'
        END as token_bucket,
        COUNT(*) as doc_count
    FROM {FULL_TABLE_NAME}
    WHERE {TEXT_COLUMN} IS NOT NULL
    GROUP BY 1
    ORDER BY 
        CASE 
            WHEN token_bucket = '0-50' THEN 1
            WHEN token_bucket = '50-100' THEN 2
            WHEN token_bucket = '100-150' THEN 3
            WHEN token_bucket = '150-200' THEN 4
            WHEN token_bucket = '200-256' THEN 5
            WHEN token_bucket = '256-400 (SAFE)' THEN 6
            WHEN token_bucket = '400-512 (CAUTION)' THEN 7
            ELSE 8
        END
""")

print("Token Distribution by Bucket:")
print("-" * 50)
for row in bucket_df.collect():
    bar_length = int(row['doc_count'] / 100)  # Scale for display
    bar = "█" * bar_length
    status = "✅" if "TRUNCATED" not in row['token_bucket'] else "❌"
    print(f"{status} {row['token_bucket']:20} | {row['doc_count']:>6,} | {bar}")

display(bucket_df)


## 5. Decision Engine


In [ ]:
# Decision Engine - Determine embedding strategy
def make_decision(stats, token_stats):
    """
    Decision tree for embedding strategy based on text analysis.
    
    Returns: (strategy, confidence, reasons)
    """
    avg_tokens = int(token_stats['avg_tokens'])
    max_tokens_in_data = int(token_stats['max_tokens'])
    p95_tokens = int(token_stats['p95_tokens'])
    exceeds_max = int(token_stats['exceeds_max'])
    total_docs = int(token_stats['total_docs'])
    truncation_rate = (exceeds_max / total_docs) * 100
    
    reasons = []
    
    # Decision Logic
    if max_tokens_in_data <= SAFE_TOKEN_THRESHOLD:
        strategy = "EMBED_FULL_TEXT"
        confidence = "HIGH"
        reasons.append(f"✅ Max tokens ({max_tokens_in_data}) well under safe threshold ({SAFE_TOKEN_THRESHOLD})")
        reasons.append(f"✅ No risk of truncation")
        reasons.append(f"✅ Full semantic meaning preserved for all documents")
        
    elif max_tokens_in_data <= MAX_TOKENS:
        strategy = "EMBED_FULL_TEXT"
        confidence = "MEDIUM"
        reasons.append(f"✅ Max tokens ({max_tokens_in_data}) within model limit ({MAX_TOKENS})")
        reasons.append(f"⚠️ Some documents approach the limit - monitor for edge cases")
        reasons.append(f"✅ Chunking adds unnecessary complexity for this data")
        
    elif truncation_rate < 1.0:
        strategy = "EMBED_FULL_TEXT"
        confidence = "MEDIUM"
        reasons.append(f"⚠️ {truncation_rate:.2f}% of documents exceed {MAX_TOKENS} tokens")
        reasons.append(f"✅ Truncation affects < 1% - acceptable for most use cases")
        reasons.append(f"💡 Consider: Truncated docs lose tail content; okay if key info is at start")
        
    elif truncation_rate < 5.0:
        strategy = "CONSIDER_CHUNKING"
        confidence = "MEDIUM"
        reasons.append(f"⚠️ {truncation_rate:.2f}% of documents will be truncated")
        reasons.append(f"⚠️ P95 tokens: {p95_tokens} (many docs are long)")
        reasons.append(f"💡 Evaluate: Is the truncated content important for retrieval?")
        reasons.append(f"💡 If yes → implement chunking; If no → embed full text")
        
    else:
        strategy = "CHUNK_REQUIRED"
        confidence = "HIGH"
        reasons.append(f"❌ {truncation_rate:.2f}% of documents exceed model limit")
        reasons.append(f"❌ Significant information loss without chunking")
        reasons.append(f"📋 Recommended: Chunk at {CHUNK_SIZE} tokens with {CHUNK_OVERLAP} overlap")
        reasons.append(f"📋 This will create ~{int(avg_tokens / CHUNK_SIZE) + 1} chunks per document on average")
    
    return strategy, confidence, reasons


# Run decision engine
strategy, confidence, reasons = make_decision(stats, token_stats)

print("=" * 70)
print("🎯 EMBEDDING STRATEGY RECOMMENDATION")
print("=" * 70)
print(f"\n  Strategy:   {strategy}")
print(f"  Confidence: {confidence}")
print(f"\n  Reasoning:")
for reason in reasons:
    print(f"    {reason}")


## 6. Strategy Comparison

| Strategy | When to Use | Pros | Cons |
|----------|-------------|------|------|
| **EMBED_FULL_TEXT** | Documents < 400 tokens | Simple, complete semantics, fast queries | Limited to short docs |
| **EMBED_WITH_TRUNCATION** | Most docs < 512, few exceed | Simple, works for most cases | Loses tail content on long docs |
| **CHUNKING** | Documents > 512 tokens | Complete coverage | Complex, more storage, need to merge results |
| **HIERARCHICAL** | Very long documents (1000s of tokens) | Best retrieval for long docs | Most complex to implement |


## 7. Sample Long Documents (if any)


In [ ]:
# Show longest documents (potential truncation candidates)
print("Top 5 Longest Documents:")
print("-" * 70)

long_docs_df = spark.sql(f"""
    SELECT 
        LENGTH({TEXT_COLUMN}) as char_count,
        ROUND(LENGTH({TEXT_COLUMN}) / {CHARS_PER_TOKEN}, 0) as est_tokens,
        SUBSTRING({TEXT_COLUMN}, 1, 100) as preview,
        SUBSTRING({TEXT_COLUMN}, -100, 100) as tail_preview
    FROM {FULL_TABLE_NAME}
    WHERE {TEXT_COLUMN} IS NOT NULL
    ORDER BY LENGTH({TEXT_COLUMN}) DESC
    LIMIT 5
""")

for i, row in enumerate(long_docs_df.collect(), 1):
    print(f"\n[{i}] {int(row['char_count'])} chars (~{int(row['est_tokens'])} tokens)")
    print(f"    Start: \"{row['preview']}...\"")
    print(f"    End:   \"...{row['tail_preview']}\"")
    if row['est_tokens'] > MAX_TOKENS:
        lost_tokens = int(row['est_tokens'] - MAX_TOKENS)
        print(f"    ⚠️ Would lose ~{lost_tokens} tokens if truncated")


## 8. Implementation Guidance

Based on the analysis above, here's how to proceed:


In [ ]:
# Generate implementation guidance based on decision
print("=" * 70)
print("📋 IMPLEMENTATION STEPS")
print("=" * 70)

if strategy == "EMBED_FULL_TEXT":
    print("""
┌─────────────────────────────────────────────────────────────────────┐
│  EMBED FULL TEXT - Recommended Implementation                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. Create Vector Search Index in Databricks UI:                    │
│     • Navigate to Catalog → Table → Create Vector Search Index      │
│     • Select "Delta Sync" for automatic updates                     │
│     • Choose "Managed Embeddings"                                   │
│     • Set embedding column to: {TEXT_COLUMN}                        │
│     • Add filter columns as needed                                  │
│                                                                     │
│  2. No preprocessing required:                                      │
│     • Your text fits within model limits                            │
│     • Full semantic meaning will be preserved                       │
│                                                                     │
│  3. Query directly:                                                 │
│     SELECT * FROM VECTOR_SEARCH(                                    │
│       index => 'your_catalog.schema.index_name',                    │
│       query => 'your search query',                                 │
│       num_results => 10                                             │
│     )                                                               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""".format(TEXT_COLUMN=TEXT_COLUMN))

elif strategy == "CONSIDER_CHUNKING":
    print("""
┌─────────────────────────────────────────────────────────────────────┐
│  EVALUATE CHUNKING - Decision Required                              │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  Questions to answer:                                               │
│                                                                     │
│  1. Where is the important information in your documents?           │
│     • Beginning → Full text OK (truncation acceptable)              │
│     • Throughout → Consider chunking                                │
│     • End → Definitely chunk (tail will be lost)                    │
│                                                                     │
│  2. What's the cost of missing information?                         │
│     • Low (e.g., nice-to-have context) → Full text OK               │
│     • High (e.g., key facts anywhere) → Chunk                       │
│                                                                     │
│  3. Query patterns:                                                 │
│     • Users search for concepts at start → Full text OK             │
│     • Users search for any detail → Chunk                           │
│                                                                     │
│  See Section 9 for chunking implementation if needed.               │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")

else:  # CHUNK_REQUIRED
    print("""
┌─────────────────────────────────────────────────────────────────────┐
│  CHUNKING REQUIRED - Implementation Steps                           │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. Create chunked table:                                           │
│     • See Section 9 for chunking code                               │
│     • Add chunk_id, parent_doc_id columns                           │
│     • Store chunk position for result ordering                      │
│                                                                     │
│  2. Create Vector Search Index on chunked table:                    │
│     • Embedding column: chunk_text                                  │
│     • Include parent_doc_id for grouping results                    │
│                                                                     │
│  3. Query with result aggregation:                                  │
│     • Search returns chunks, not full docs                          │
│     • Group by parent_doc_id to reconstruct context                 │
│     • Consider MMR (Maximal Marginal Relevance) for diversity       │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
""")


## 9. Chunking Reference Implementation (If Needed)

Only use this if the decision engine recommends chunking. This code is provided as a reference.


In [ ]:
# REFERENCE: Chunking implementation (only run if chunking is recommended)
# Uncomment and modify as needed

"""
from pyspark.sql.functions import col, udf, explode, monotonically_increasing_id
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType

# Chunking function
def chunk_text(text, chunk_size=400, overlap=50):
    '''
    Split text into overlapping chunks.
    
    Args:
        text: Input text to chunk
        chunk_size: Target tokens per chunk (approx)
        overlap: Token overlap between chunks
    
    Returns:
        List of (chunk_index, chunk_text) tuples
    '''
    if not text:
        return [(0, "")]
    
    # Approximate: 4 chars per token
    char_chunk_size = chunk_size * 4
    char_overlap = overlap * 4
    
    chunks = []
    start = 0
    chunk_idx = 0
    
    while start < len(text):
        end = start + char_chunk_size
        
        # Try to break at sentence boundary
        if end < len(text):
            # Look for sentence end (.!?) within last 20% of chunk
            search_start = int(end - char_chunk_size * 0.2)
            for i in range(end, search_start, -1):
                if text[i] in '.!?' and (i + 1 >= len(text) or text[i + 1] == ' '):
                    end = i + 1
                    break
        
        chunk_text = text[start:end].strip()
        if chunk_text:
            chunks.append((chunk_idx, chunk_text))
            chunk_idx += 1
        
        start = end - char_overlap
    
    return chunks

# Register UDF
chunk_schema = ArrayType(StructType([
    StructField("chunk_idx", IntegerType()),
    StructField("chunk_text", StringType())
]))
chunk_udf = udf(lambda text: chunk_text(text, CHUNK_SIZE, CHUNK_OVERLAP), chunk_schema)

# Create chunked table
chunked_df = (
    spark.table(FULL_TABLE_NAME)
    .withColumn("chunks", chunk_udf(col(TEXT_COLUMN)))
    .withColumn("chunk", explode("chunks"))
    .select(
        col("*"),
        col("chunk.chunk_idx").alias("chunk_index"),
        col("chunk.chunk_text").alias("chunk_text")
    )
    .drop("chunks", "chunk")
)

# Save chunked table
CHUNKED_TABLE = f"{CATALOG}.{SCHEMA}.{TABLE_NAME}_chunked"
chunked_df.write.mode("overwrite").saveAsTable(CHUNKED_TABLE)

print(f"Created chunked table: {CHUNKED_TABLE}")
print(f"Original docs: {spark.table(FULL_TABLE_NAME).count()}")
print(f"Total chunks:  {spark.table(CHUNKED_TABLE).count()}")
"""

print("⬆️ Chunking code is commented out (reference only)")
print("   Uncomment and run if the decision engine recommends chunking")


## 10. Summary


---

# Part 2: Filter Column Selection

## Why Filter Columns Matter

Filter columns enable **pre-filtering** before similarity search:
- **Query performance**: Reduce search space before vector comparison
- **Result relevance**: Only search within relevant subsets
- **Cost efficiency**: Less compute for embedding comparisons

## Filter Column Decision Framework

```
                    ┌─────────────────────────────────┐
                    │  Analyze Each Column            │
                    └───────────────┬─────────────────┘
                                    │
                    ┌───────────────▼─────────────────┐
                    │  Count Distinct Values          │
                    │  (Cardinality)                  │
                    └───────────────┬─────────────────┘
                                    │
        ┌───────────────────────────┼───────────────────────────┐
        │                           │                           │
┌───────▼───────┐           ┌───────▼───────┐           ┌───────▼───────┐
│   < 20        │           │   20 - 100    │           │    > 100      │
│   values      │           │   values      │           │   values      │
└───────┬───────┘           └───────┬───────┘           └───────┬───────┘
        │                           │                           │
        ▼                           ▼                           ▼
┌───────────────┐           ┌───────────────┐           ┌───────────────┐
│ ✅ EXCELLENT  │           │ ⚠️ CONSIDER   │           │ ❌ AVOID      │
│ Filter Column │           │ If frequently │           │ Use joins or  │
│               │           │ filtered on   │           │ post-filter   │
└───────────────┘           └───────────────┘           └───────────────┘
```

## Key Principles

| Principle | Good Filter | Bad Filter |
|-----------|-------------|------------|
| **Cardinality** | Low (< 100 distinct values) | High (unique per row) |
| **Usage** | Frequently filtered in queries | Rarely or never filtered |
| **Selectivity** | Significantly reduces result set | Minimal impact |
| **Type** | Categorical, enum-like | Free text, continuous numeric |


## 11. Column Cardinality Analysis


In [ ]:
# =============================================================================
# COLUMN CARDINALITY ANALYSIS
# =============================================================================
# Analyze all columns to determine suitability as filter columns

from pyspark.sql.types import StringType, IntegerType, BooleanType, DoubleType

# Get table schema
schema = spark.table(FULL_TABLE_NAME).schema
total_rows = spark.sql(f"SELECT COUNT(*) as cnt FROM {FULL_TABLE_NAME}").collect()[0]['cnt']

print("=" * 80)
print("COLUMN CARDINALITY ANALYSIS")
print("=" * 80)
print(f"\nTable: {FULL_TABLE_NAME}")
print(f"Total Rows: {total_rows:,}")
print("\nAnalyzing columns...\n")

# Analyze each column
column_analysis = []

for field in schema:
    col_name = field.name
    col_type = str(field.dataType)
    
    # Skip the embedding column and primary key (we know these already)
    is_embedding = col_name == TEXT_COLUMN
    
    # Get distinct count
    distinct_count = spark.sql(f"""
        SELECT COUNT(DISTINCT `{col_name}`) as distinct_count 
        FROM {FULL_TABLE_NAME}
    """).collect()[0]['distinct_count']
    
    # Calculate cardinality ratio
    cardinality_ratio = distinct_count / total_rows if total_rows > 0 else 0
    
    # Determine recommendation
    if is_embedding:
        recommendation = "EMBEDDING"
        reason = "This is the embedding source column"
    elif distinct_count == total_rows:
        recommendation = "❌ AVOID"
        reason = "Unique per row (likely PK or ID)"
    elif distinct_count <= 10:
        recommendation = "✅ EXCELLENT"
        reason = "Very low cardinality, ideal for filtering"
    elif distinct_count <= 50:
        recommendation = "✅ GOOD"
        reason = "Low cardinality, good filter candidate"
    elif distinct_count <= 100:
        recommendation = "⚠️ CONSIDER"
        reason = "Moderate cardinality, use if frequently queried"
    elif cardinality_ratio > 0.5:
        recommendation = "❌ AVOID"
        reason = "High cardinality (>50% unique)"
    else:
        recommendation = "⚠️ MAYBE"
        reason = f"Many values ({distinct_count}), evaluate usage patterns"
    
    column_analysis.append({
        'column': col_name,
        'type': col_type,
        'distinct': distinct_count,
        'ratio': cardinality_ratio,
        'recommendation': recommendation,
        'reason': reason
    })

# Sort by distinct count (lowest first = best filters)
column_analysis.sort(key=lambda x: (
    0 if x['recommendation'].startswith('✅') else 
    1 if x['recommendation'].startswith('⚠️') else 2,
    x['distinct']
))

# Display results
print(f"{'Column':<25} {'Type':<15} {'Distinct':>10} {'Ratio':>8} {'Recommendation':<15}")
print("-" * 80)

for col in column_analysis:
    ratio_str = f"{col['ratio']:.1%}" if col['ratio'] < 1 else "100%"
    print(f"{col['column']:<25} {col['type'][:14]:<15} {col['distinct']:>10,} {ratio_str:>8} {col['recommendation']:<15}")

print("\n" + "-" * 80)


## 12. Filter Column Recommendations


In [ ]:
# =============================================================================
# FILTER COLUMN RECOMMENDATIONS
# =============================================================================

# Categorize columns
excellent_filters = [c for c in column_analysis if c['recommendation'] == '✅ EXCELLENT']
good_filters = [c for c in column_analysis if c['recommendation'] == '✅ GOOD']
consider_filters = [c for c in column_analysis if c['recommendation'].startswith('⚠️')]
avoid_filters = [c for c in column_analysis if c['recommendation'] == '❌ AVOID']
embedding_col = [c for c in column_analysis if c['recommendation'] == 'EMBEDDING']

print("=" * 70)
print("🎯 FILTER COLUMN RECOMMENDATIONS")
print("=" * 70)

# Excellent filters
if excellent_filters:
    print("\n✅ EXCELLENT Filter Columns (use these!):")
    print("-" * 50)
    for col in excellent_filters:
        print(f"   • {col['column']:<25} ({col['distinct']} values)")
        print(f"     {col['reason']}")

# Good filters
if good_filters:
    print("\n✅ GOOD Filter Columns (recommended):")
    print("-" * 50)
    for col in good_filters:
        print(f"   • {col['column']:<25} ({col['distinct']} values)")
        print(f"     {col['reason']}")

# Consider
if consider_filters:
    print("\n⚠️ CONSIDER These (if frequently used in queries):")
    print("-" * 50)
    for col in consider_filters:
        print(f"   • {col['column']:<25} ({col['distinct']} values)")
        print(f"     {col['reason']}")

# Avoid
if avoid_filters:
    print("\n❌ AVOID as Filter Columns:")
    print("-" * 50)
    for col in avoid_filters[:5]:  # Show first 5
        print(f"   • {col['column']:<25} ({col['distinct']} values)")
        print(f"     {col['reason']}")
    if len(avoid_filters) > 5:
        print(f"   ... and {len(avoid_filters) - 5} more high-cardinality columns")

print("\n" + "=" * 70)


## 13. Sample Value Distribution (Top Filter Candidates)


In [ ]:
# =============================================================================
# SHOW VALUE DISTRIBUTION FOR TOP FILTER CANDIDATES
# =============================================================================

# Get top filter candidates (excellent + good)
top_candidates = excellent_filters + good_filters

if top_candidates:
    print("VALUE DISTRIBUTION FOR RECOMMENDED FILTER COLUMNS")
    print("=" * 70)
    
    for col_info in top_candidates[:6]:  # Show up to 6 columns
        col_name = col_info['column']
        print(f"\n📊 {col_name} ({col_info['distinct']} distinct values):")
        print("-" * 50)
        
        # Get value distribution
        dist_df = spark.sql(f"""
            SELECT 
                `{col_name}` as value,
                COUNT(*) as count,
                ROUND(COUNT(*) * 100.0 / {total_rows}, 1) as pct
            FROM {FULL_TABLE_NAME}
            GROUP BY `{col_name}`
            ORDER BY count DESC
            LIMIT 10
        """)
        
        for row in dist_df.collect():
            value = row['value'] if row['value'] is not None else "(NULL)"
            bar_len = int(row['pct'] / 2)
            bar = "█" * bar_len
            print(f"   {str(value):<20} | {row['count']:>6,} ({row['pct']:>5.1f}%) {bar}")
        
        if col_info['distinct'] > 10:
            print(f"   ... and {col_info['distinct'] - 10} more values")
else:
    print("No excellent or good filter candidates found.")


## 14. Generated Filter Column Configuration


In [ ]:
# =============================================================================
# GENERATE FILTER COLUMN CONFIGURATION
# =============================================================================

# Build recommended filter columns list
recommended_filters = []

# Add all excellent filters
for col in excellent_filters:
    recommended_filters.append(col['column'])

# Add good filters (up to a reasonable total)
for col in good_filters:
    if len(recommended_filters) < 8:  # Keep it manageable
        recommended_filters.append(col['column'])

print("=" * 70)
print("📋 RECOMMENDED FILTER COLUMNS CONFIGURATION")
print("=" * 70)

print(f"\nRecommended filter columns ({len(recommended_filters)}):")
print("-" * 50)
for i, col in enumerate(recommended_filters, 1):
    col_info = next(c for c in column_analysis if c['column'] == col)
    print(f"  {i}. {col:<25} ({col_info['distinct']} values)")

# Generate code snippet
filter_list = ", ".join([f'"{c}"' for c in recommended_filters])

print("\n" + "=" * 70)
print("📝 COPY-PASTE CODE FOR NOTEBOOK 04")
print("=" * 70)
print(f"""
# Filter columns for Vector Search index
FILTER_COLUMNS = [{filter_list}]

# Or as comma-separated string for widget:
# "{','.join(recommended_filters)}"
""")

# Store for use in summary
RECOMMENDED_FILTER_COLUMNS = recommended_filters


## 15. Complete Configuration Summary


In [ ]:
# =============================================================================
# COMPLETE VECTOR SEARCH CONFIGURATION SUMMARY
# =============================================================================

print("=" * 80)
print("🎯 COMPLETE VECTOR SEARCH CONFIGURATION SUMMARY")
print("=" * 80)

print(f"""
┌─────────────────────────────────────────────────────────────────────────────┐
│  TABLE INFORMATION                                                          │
├─────────────────────────────────────────────────────────────────────────────┤
│  Table:              {FULL_TABLE_NAME:<50} │
│  Total Rows:         {total_rows:>10,}                                           │
│  Total Columns:      {len(schema):>10}                                           │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│  PART 1: EMBEDDING STRATEGY                                                 │
├─────────────────────────────────────────────────────────────────────────────┤
│  Decision:           {strategy:<50} │
│  Confidence:         {confidence:<50} │
│  Embedding Column:   {TEXT_COLUMN:<50} │
│  Embedding Model:    {EMBEDDING_MODEL:<50} │
└─────────────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────────────┐
│  PART 2: FILTER COLUMNS                                                     │
├─────────────────────────────────────────────────────────────────────────────┤
│  Recommended Filters: {len(RECOMMENDED_FILTER_COLUMNS):<49} │
│  Columns:            {', '.join(RECOMMENDED_FILTER_COLUMNS[:4]):<50} │
│                      {', '.join(RECOMMENDED_FILTER_COLUMNS[4:8]) if len(RECOMMENDED_FILTER_COLUMNS) > 4 else '':<50} │
└─────────────────────────────────────────────────────────────────────────────┘
""")

# Print the configuration for notebook 04
print("=" * 80)
print("📋 READY-TO-USE CONFIGURATION FOR NOTEBOOK 04")
print("=" * 80)
print(f'''
# Vector Search Index Configuration
CATALOG = "{CATALOG}"
SCHEMA = "{SCHEMA}"
SOURCE_TABLE = "{TABLE_NAME}"
INDEX_NAME = "{TABLE_NAME}_idx"
EMBEDDING_COLUMN = "{TEXT_COLUMN}"
EMBEDDING_MODEL = "{EMBEDDING_MODEL}"
FILTER_COLUMNS = {RECOMMENDED_FILTER_COLUMNS}

# Widget parameter string:
# filter_columns = "{','.join(RECOMMENDED_FILTER_COLUMNS)}"
''')

print("=" * 80)
print("✅ Analysis complete! Use this configuration in notebook 04-vector-search-create.ipynb")
print("=" * 80)


In [ ]:
# Final Summary
print("=" * 70)
print("📊 VECTOR SEARCH IMPLEMENTATION SUMMARY")
print("=" * 70)

print(f"""
┌─────────────────────────────────────────────────────────────────────┐
│  DATA ANALYZED                                                      │
├─────────────────────────────────────────────────────────────────────┤
│  Table:           {FULL_TABLE_NAME:<40}      │
│  Text Column:     {TEXT_COLUMN:<40}      │
│  Total Documents: {int(token_stats['total_docs']):>10,}                                   │
│  Embedding Model: {EMBEDDING_MODEL:<40}      │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│  TEXT STATISTICS                                                    │
├─────────────────────────────────────────────────────────────────────┤
│  Avg Characters:  {int(stats['avg_chars']):>10,}                                   │
│  Max Characters:  {int(stats['max_chars']):>10,}                                   │
│  Avg Tokens:      {int(token_stats['avg_tokens']):>10,} (estimated)                      │
│  Max Tokens:      {int(token_stats['max_tokens']):>10,} (estimated)                      │
│  Model Limit:     {MAX_TOKENS:>10,}                                   │
└─────────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────────┐
│  RECOMMENDATION                                                     │
├─────────────────────────────────────────────────────────────────────┤
│  Strategy:        {strategy:<40}      │
│  Confidence:      {confidence:<40}      │
└─────────────────────────────────────────────────────────────────────┘
""")

# Quick decision summary
if strategy == "EMBED_FULL_TEXT":
    print("✅ VERDICT: Embed the full text. No chunking needed.")
    print("   Your documents are short enough that full semantic meaning is preserved.")
elif strategy == "CONSIDER_CHUNKING":
    print("⚠️ VERDICT: Evaluate your specific use case.")
    print("   Some documents may be truncated. Decide based on where important info lives.")
else:
    print("📋 VERDICT: Implement chunking before creating the index.")
    print("   Too many documents exceed the model limit for full-text embedding.")
